In [ ]:
from atproto import Client
from datetime import datetime, timedelta, timezone, date
from collections import Counter
from typing import Dict, List, Iterable, Optional
import os, csv, time, requests

# ---------- CONFIG / LOGIN ----------
client = Client()
client.login("YOUR USERNAME.bsky.social", "YOUR PASSWORD")  # for production, consider env vars

# Where to save results (macOS)
SAVE_DIR = "/Users/nihan/Desktop/bluesky"

# HTTP fallback base (public AppView)
APPVIEW_BASE = "https://public.api.bsky.app"

# ---------- COMPANIES & ALIASES ----------
fortune_20 ={
    1: "Walmart",
    2: "Amazon",
    3: "Apple",
    4: "CVS",
    5: "Tesla",
    6: "Google",
    7: "Meta",
    8: "JPMorgan",
    9: "Costco",
    10: "Kroger",
    11: "Walgreens",
    12: "Target",
    13: "UPS",
    14: "Centene",
    15: "Cigna",
    16: "Microsoft",
    17: "Verizon",
    18: "IBM",
    19: "UnitedHealth",
    20: "ExxonMobil",
    21: "McKesson",
    22: "Chevron",
    23: "Cardinal Health",
    24: "Home Depot",
    25: "Walgreens Boots Alliance",
    26: "Marathon Petroleum",
    27: "Cencora",
    28: "Ford",
    29: "Citigroup",
    30: "Dell Technologies",
    31: "General Motors",
    32: "Nvidia",
    33: "Pfizer",
    34: "Procter & Gamble",
    35: "Comcast",
    36: "Johnson & Johnson",
    37: "Wells Fargo",
    38: "Bank of America",
    39: "AbbVie",
    40: "Starbucks",
    41: "Cisco",
    42: "AT&T",
    43: "PepsiCo",
    44: "Intel",
    45: "The Walt Disney Company",
    46: "Boeing",
    47: "Goldman Sachs",
    48: "Morgan Stanley",
    49: "Honeywell",
    50: "Salesforce"
}

COMPANIES = list(fortune_20.values())

# Add this dict (aliases + ticker forms). Multi-word terms are auto-quoted by your _normalize_term.
COMPANY_ALIASES = {
    "Walmart": [
        "Walmart", "#Walmart", "WMT", "$WMT", "Doug McMillon", "Walmrt", "Walmrt#", "WMLT"
    ],
    "Amazon": [
        "Amazon", "AMZN", "$AMZN", "#AMZN", "Amazon.com", "Andy Jassy", "Amzon", "Aamzon", "AMZON", "Jeff Bezos"
    ],
    "Apple": [
        "Apple", "AAPL", "$AAPL", "#AAPL", "Apple Inc", "Tim Cook", "Aple", "Appl", "AAPLE"
    ],
    "CVS": [
        "CVS", "CVS Health", "CVS Pharmacy", "CVS Health Corp", "CVS Health Corporation", "$CVS", "#CVS", "CVS", "David Joyner", "CVSS", "CV Health"
    ],
    "Tesla": [
        "Tesla", "TSLA", "$TSLA", "#TSLA", "Tesla Motors", "Elon Musk", "Tesl", "Tsla", "TSLAA"
    ],
    "Google": [
        "Google", "Alphabet Inc", "GOOGL", "$GOOGL", "#GOOGL", "Sundar Pichai", "Gooogle", "Gogle", "GOGL"
    ],
    "Meta": [
        "Meta", "Meta Platforms", "META", "$META", "#META", "Facebook", "Mark Zuckerberg", "Metta", "Mett", "Metaa"
    ],
    "JPMorgan": [
        "JPMorgan", "JP Morgan", "JPMorgan Chase", "JPMorgan Chase & Co.", "JPM", "$JPM", "#JPM", "Chase", "#Chase", "Jamie Dimon", "JPMO", "JP Morgan Chase"
    ],
    "Costco": [
        "Costco", "Costco Wholesale", "COST", "$COST", "Ron Vachris", "Costcoo", "Costkco"
    ],
    "Kroger": [
        "Kroger", "The Kroger Co.", "KR", "$KR", "Rodney McMullen", "Kroger Co", "Krogar"
    ],
    "Walgreens": [
        "Walgreens", "WBA", "$WBA", "Tim Wentworth", "Walgreen", "Walgrens"
    ],
    "Target": [
        "Target", "TGT", "$TGT", "Brian Cornell", "Targert", "Traget"
    ],
    "UPS": [
        "UPS", "United Parcel Service", "$UPS", "#UPS", "Carol Tomé", "Carol Tome", "UPZ", "Upes"
    ],
    "Centene": [
        "Centene", "Centene Corp", "Centene Corporation", "CNC", "$CNC", "Sarah London", "Centain", "Centnee"
    ],
    "Cigna": [
        "Cigna", "The Cigna Group", "CI", "$CI", "David Cordani", "Cigna Group", "Cigna Health"
    ],
    "Microsoft": [
        "Microsoft", "MSFT", "$MSFT", "#MSFT", "Satya Nadella", "Microsof", "Microsft", "Bill Gates"
    ],
    "Verizon": [
        "Verizon", "Verizon Communications", "VZ", "$VZ", "Hans Vestberg", "Verizon Comm", "Verisone"
    ],
    "IBM": [
        "IBM", "International Business Machines", "$IBM", "#IBM", "Arvind Krishna", "IBMM", "IBM Corp"
    ],
    "UnitedHealth": [
        "UnitedHealth", "UnitedHealth Group", "United Healthcare", "UnitedHealthcare", "UNH", "$UNH", "#UNH", "Stephen Hemsley", "United Health"
    ],
    "ExxonMobil": [
        "ExxonMobil", "Exxon Mobil Corporation", "XOM", "$XOM", "Darren Woods", "Exxon Mobile", "ExonnMobil"
    ],
    "McKesson": [
        "McKesson", "McKesson Corp", "McKesson Corporation", "MCK", "$MCK", "Brian Tyler", "Mckesson Corporation", "McKessonCo"
    ],
    "Chevron": [
        "Chevron", "CVX", "$CVX", "Michael Wirth", "Chevorn", "Chevero"
    ],
    "Cardinal Health": [
        "Cardinal Health", "CAH", "$CAH", "Jason Hollar", "Cardnel Health"
    ],
    "Home Depot": [
        "Home Depot", "The Home Depot", "HD", "$HD", "Ted Decker", "HomeDeopt", "Hom Depot"
    ],
    "Walgreens Boots Alliance": [
        "Walgreens Boots Alliance", "WBA", "$WBA", "Tim Wentworth", "Walgreens Boots Alianace"
    ],
    "Marathon Petroleum": [
        "Marathon Petroleum", "MPC", "$MPC", "Michael Hennigan", "Marathon Petroelum"
    ],
    "Cencora": [
        "Cencora", "COR", "$COR", "Robert P. Mauch", "Cencora Inc"
    ],
    "Ford": [
        "Ford", "Ford Motor Company", "F", "$F", "Jim Farley", "Frod", "Ford Co"
    ],
    "Citigroup": [
        "Citigroup", "C", "$C", "Jane Fraser", "Citigroup Inc"
    ],
    "Dell Technologies": [
        "Dell Technologies", "DELL", "$DELL", "Michael Dell", "Del Technologies"
    ],
    "General Motors": [
        "General Motors", "GM", "$GM", "Mary Barra", "General Moters"
    ],
    "Nvidia": [
        "Nvidia", "NVDA", "$NVDA", "Jensen Huang", "Nvidiaa"
    ],
    "Pfizer": [
        "Pfizer", "PFE", "$PFE", "Albert Bourla", "Pfizzer"
    ],
    "Procter & Gamble": [
        "Procter & Gamble", "P&G", "PG", "$PG", "Jon Moeller", "Procter and Gamble", "Procter & Gample"
    ],
    "Comcast": [
        "Comcast", "CMCSA", "$CMCSA", "Brian Roberts", "Commcast"
    ],
    "Johnson & Johnson": [
        "Johnson & Johnson", "J&J", "JNJ", "$JNJ", "Joaquin Duato", "Johnsons & Johnson"
    ],
    "Wells Fargo": [
        "Wells Fargo", "WFC", "$WFC", "Charles Scharf", "Wells Fago"
    ],
    "Bank of America": [
        "Bank of America", "BAC", "$BAC", "Brian Moynihan", "Bank of Amercia"
    ],
    "AbbVie": [
        "AbbVie", "ABBV", "$ABBV", "Richard Gonzalez", "Abbvie"
    ],
    "Starbucks": [
        "Starbucks", "SBUX", "$SBUX", "Kevin Johnson", "Starbuks", "Starbucs", "Starbuck", "Starbux"
    ],
    "Cisco": [
        "Cisco", "Cisco Systems", "CSCO", "$CSCO", "Chuck Robbins", "Ciscco"
    ],
    "AT&T": [
        "AT&T", "T", "$T", "John Stankey", "AT and T"
    ],
    "PepsiCo": [
        "PepsiCo", "PEP", "$PEP", "Ramon Laguarta", "Pepsi Co"
    ],
    "Intel": [
        "Intel", "INTC", "$INTC", "Pat Gelsinger", "Intell"
    ],
    "The Walt Disney Company": [
        "Disney", "The Walt Disney Company", "DIS", "$DIS", "Bob Iger", "Diseny"
    ],
    "Boeing": [
        "Boeing", "BA", "$BA", "David Calhoun", "Boieng"
    ],
    "Goldman Sachs": [
        "Goldman Sachs", "GS", "$GS", "David Solomon", "Goldmann Sachs"
    ],
    "Morgan Stanley": [
        "Morgan Stanley", "MS", "$MS", "Ted Pick", "Morgan Stanly"
    ],
    "Honeywell": [
        "Honeywell", "Honeywell International", "HON", "$HON", "Vimal Kapur", "Honeywel"
    ],
    "Salesforce": [
        "Salesforce", "CRM", "$CRM", "Marc Benioff", "SalesForce", "SaleForce"
    ]
}

TICKER_ALIASES = {
    "Walmart": ["Walmart", "#Walmart", "WMT", "$WMT", "Walmrt", "Walmrt#", "WMLT"],
    "Amazon": ["Amazon", "AMZN", "$AMZN", "#AMZN", "Amazon.com", "Amzon", "Aamzon", "AMZON"],
    "Apple": ["Apple", "AAPL", "$AAPL", "#AAPL", "Apple Inc", "Aple", "Appl", "AAPLE"],
    "CVS": ["CVS", "CVS Health", "CVS Pharmacy", "CVS Health Corp", "CVS Health Corporation", "$CVS", "#CVS", "CVSS", "CV Health"],
    "Tesla": ["Tesla", "TSLA", "$TSLA", "#TSLA", "Tesla Motors", "Tesl", "Tsla", "TSLAA"],
    "Google": ["Google", "Alphabet Inc", "GOOGL", "$GOOGL", "#GOOGL", "Gooogle", "Gogle", "GOGL"],
    "Meta": ["Meta", "Meta Platforms", "META", "$META", "#META", "Facebook", "Metta", "Mett", "Metaa"],
    "JPMorgan": ["JPMorgan", "JP Morgan", "JPMorgan Chase", "JPM", "$JPM", "#JPM", "JPMO", "JP Morgan Chase"],
    "Costco": ["Costco", "Costco Wholesale", "COST", "$COST", "Costcoo", "Costkco"],
    "Kroger": ["Kroger", "The Kroger Co.", "KR", "$KR", "Kroger Co", "Krogar"],
    "Walgreens": ["Walgreens", "WBA", "$WBA", "Walgreen", "Walgrens"],
    "Target": ["Target", "TGT", "$TGT", "Targert", "Traget"],
    "UPS": ["UPS", "United Parcel Service", "$UPS", "#UPS", "UPZ", "Upes"],
    "Centene": ["Centene", "Centene Corp", "Centene Corporation", "CNC", "$CNC", "Centain", "Centnee"],
    "Cigna": ["Cigna", "The Cigna Group", "CI", "$CI", "Cigna Group", "Cigna Health"],
    "Microsoft": ["Microsoft", "MSFT", "$MSFT", "#MSFT", "Microsof", "Microsft"],
    "Verizon": ["Verizon", "Verizon Communications", "VZ", "$VZ", "Verizon Comm", "Verisone"],
    "IBM": ["IBM", "International Business Machines", "$IBM", "#IBM", "IBMM", "IBM Corp"],
    "UnitedHealth": ["UnitedHealth", "UnitedHealth Group", "United Healthcare", "UNH", "$UNH", "#UNH", "United Health"],
    "ExxonMobil": ["ExxonMobil", "Exxon Mobil Corporation", "XOM", "$XOM", "Exxon Mobile", "ExonnMobil"],
    "McKesson": ["McKesson", "McKesson Corp", "McKesson Corporation", "MCK", "$MCK", "Mckesson Corporation", "McKessonCo"],
    "Chevron": ["Chevron", "CVX", "$CVX", "Chevorn", "Chevero"],
    "Cardinal Health": ["Cardinal Health", "CAH", "$CAH", "Cardnel Health"],
    "Home Depot": ["Home Depot", "The Home Depot", "HD", "$HD", "HomeDeopt", "Hom Depot"],
    "Walgreens Boots Alliance": ["Walgreens Boots Alliance", "WBA", "$WBA", "Walgreens Boots Alianace"],
    "Marathon Petroleum": ["Marathon Petroleum", "MPC", "$MPC", "Marathon Petroelum"],
    "Cencora": ["Cencora", "COR", "$COR", "Cencora Inc", "Robert P. Mauch"],
    "Ford": ["Ford", "Ford Motor Company", "F", "$F", "Frod", "Ford Co"],
    "Citigroup": ["Citigroup", "C", "$C", "Citigroup Inc", "Jane Fraser"],
    "Dell Technologies": ["Dell Technologies", "DELL", "$DELL", "Del Technologies"],
    "General Motors": ["General Motors", "GM", "$GM", "General Moters"],
    "Nvidia": ["Nvidia", "NVDA", "$NVDA", "Nvidiaa"],
    "Pfizer": ["Pfizer", "PFE", "$PFE", "Pfizzer"],
    "Procter & Gamble": ["Procter & Gamble", "P&G", "PG", "$PG", "Procter and Gamble", "Procter & Gample"],
    "Comcast": ["Comcast", "CMCSA", "$CMCSA", "Commcast"],
    "Johnson & Johnson": ["Johnson & Johnson", "J&J", "JNJ", "$JNJ", "Johnsons & Johnson"],
    "Wells Fargo": ["Wells Fargo", "WFC", "$WFC", "Wells Fago"],
    "Bank of America": ["Bank of America", "BAC", "$BAC", "Bank of Amercia"],
    "AbbVie": ["AbbVie", "ABBV", "$ABBV", "Abbvie"],
    "Starbucks": ["Starbucks", "SBUX", "$SBUX", "Starbuks", "Starbucs", "Starbuck", "Starbux"],
    "Cisco": ["Cisco", "Cisco Systems", "CSCO", "$CSCO", "Ciscco"],
    "AT&T": ["AT&T", "T", "$T", "AT and T"],
    "PepsiCo": ["PepsiCo", "PEP", "$PEP", "Pepsi Co"],
    "Intel": ["Intel", "INTC", "$INTC", "Intell"],
    "The Walt Disney Company": ["Disney", "The Walt Disney Company", "DIS", "$DIS", "Diseny"],
    "Boeing": ["Boeing", "BA", "$BA", "Boieng"],
    "Goldman Sachs": ["Goldman Sachs", "GS", "$GS", "Goldmann Sachs"],
    "Morgan Stanley": ["Morgan Stanley", "MS", "$MS", "Morgan Stanly"],
    "Honeywell": ["Honeywell", "Honeywell International", "HON", "$HON", "Honeywel"],
    "Salesforce": ["Salesforce", "CRM", "$CRM", "SalesForce", "SaleForce"]
}

CEO_LIST = {
    "Walmart": ["Doug McMillon"],
    "Amazon": ["Andy Jassy", "Jeff Bezos"],
    "Apple": ["Tim Cook"],
    "CVS": ["David Joyner"],
    "Tesla": ["Elon Musk"],
    "Google": ["Sundar Pichai"],
    "Meta": ["Mark Zuckerberg"],
    "JPMorgan": ["Jamie Dimon"],
    "Costco": ["Ron Vachris"],
    "Kroger": ["Rodney McMullen"],
    "Walgreens": ["Tim Wentworth"],
    "Target": ["Brian Cornell"],
    "UPS": ["Carol Tomé", "Carol Tome"],
    "Centene": ["Sarah London"],
    "Cigna": ["David Cordani"],
    "Microsoft": ["Satya Nadella", "Bill Gates"],
    "Verizon": ["Hans Vestberg"],
    "IBM": ["Arvind Krishna"],
    "UnitedHealth": ["Stephen Hemsley"],
    "ExxonMobil": ["Darren Woods"],
    "McKesson": ["Brian Tyler"],
    "Chevron": ["Michael Wirth"],
    "Cardinal Health": ["Jason Hollar"],
    "Home Depot": ["Ted Decker"],
    "Walgreens Boots Alliance": ["Tim Wentworth"],
    "Marathon Petroleum": ["Michael Hennigan"],
    "Cencora": ["Robert P. Mauch"],
    "Ford": ["Jim Farley"],
    "Citigroup": ["Jane Fraser"],
    "Dell Technologies": ["Michael Dell"],
    "General Motors": ["Mary Barra"],
    "Nvidia": ["Jensen Huang"],
    "Pfizer": ["Albert Bourla"],
    "Procter & Gamble": ["Jon Moeller"],
    "Comcast": ["Brian Roberts"],
    "Johnson & Johnson": ["Joaquin Duato"],
    "Wells Fargo": ["Charles Scharf"],
    "Bank of America": ["Brian Moynihan"],
    "AbbVie": ["Richard Gonzalez"],
    "Starbucks": ["Kevin Johnson"],
    "Cisco": ["Chuck Robbins"],
    "AT&T": ["John Stankey"],
    "PepsiCo": ["Ramon Laguarta"],
    "Intel": ["Pat Gelsinger"],
    "The Walt Disney Company": ["Bob Iger"],
    "Boeing": ["David Calhoun"],
    "Goldman Sachs": ["David Solomon"],
    "Morgan Stanley": ["Ted Pick"],
    "Honeywell": ["Vimal Kapur"],
    "Salesforce": ["Marc Benioff"]
}

# ---------- HELPERS ----------
def _parse_iso_created_at(maybe_iso: str) -> Optional[datetime]:
    if not maybe_iso:
        return None
    try:
        return datetime.fromisoformat(maybe_iso.replace("Z", "+00:00"))
    except Exception:
        return None

def _post_created_at(post) -> Optional[datetime]:
    try:
        rec = getattr(post, "record", None)
        if rec:
            for name in ("created_at", "createdAt"):
                val = getattr(rec, name, None)
                if val:
                    dt = _parse_iso_created_at(val)
                    if dt:
                        return dt
    except Exception:
        pass
    for name in ("created_at", "createdAt"):
        val = getattr(post, name, None)
        if val:
            dt = _parse_iso_created_at(val)
            if dt:
                return dt
    return None

def _get_post_uri(obj) -> Optional[str]:
    """Return a stable unique id for dedupe (SDK: .uri, HTTP: ['uri'])."""
    if isinstance(obj, dict):
        return obj.get("uri")
    return getattr(obj, "uri", None)

def _date_range_last_n_days(days: int, today: Optional[date] = None) -> List[date]:
    if today is None:
        today = datetime.now(timezone.utc).date()
    return [(today - timedelta(days=i)) for i in range(days, 0, -1)]

# NEW: explicit date range helper (inclusive start, exclusive end)
def _date_range_between(start_date: date, end_date: date) -> List[date]:
    """Return all UTC dates between start_date and end_date (inclusive of start, exclusive of end)."""
    delta = (end_date - start_date).days
    return [(start_date + timedelta(days=i)) for i in range(delta)]

def _normalize_term(term: str) -> str:
    """Wrap multi-word terms in quotes if not already."""
    t = term.strip()
    if " " in t and not (t.startswith('"') and t.endswith('"')):
        t = f'"{t}"'
    return t

def _q_for_day(base_q: str, day_utc: date) -> str:
    """Inject UTC window into q so the API only returns that day."""
    day1 = day_utc.isoformat()
    day2 = (day_utc + timedelta(days=1)).isoformat()
    return f'{base_q} since:{day1} until:{day2}'

# ---------- RESILIENT SEARCH ----------
_RETRYABLE_HTTP = {429, 500, 502, 503, 504}

def _sdk_search(q: str, limit: int, cursor: Optional[str]) -> object:
    params = {"q": q, "limit": limit}
    if cursor:
        params["cursor"] = cursor
    return client.app.bsky.feed.search_posts(params)

def _http_search(q: str, limit: int, cursor: Optional[str]) -> dict:
    params = {"q": q, "limit": limit}
    if cursor:
        params["cursor"] = cursor
    r = requests.get(f"{APPVIEW_BASE}/xrpc/app.bsky.feed.searchPosts", params=params, timeout=30)
    if r.status_code in _RETRYABLE_HTTP:
        r.raise_for_status()
    r.raise_for_status()
    return r.json()

def _search_resilient(q: str, limit: int, cursor: Optional[str],
                      sdk_tries: int = 3, http_tries: int = 3,
                      backoff_base: float = 1.5):
    """
    Try SDK with exponential backoff; on failure, fallback to HTTP with backoff.
    Returns (posts, next_cursor, used_http: bool).
    Posts are either SDK objects or dicts.
    """
    # SDK
    attempt = 0
    while attempt < sdk_tries:
        attempt += 1
        try:
            resp = _sdk_search(q, limit, cursor)
            posts = getattr(resp, "posts", []) or []
            next_cur = getattr(resp, "cursor", None)
            return posts, next_cur, False
        except Exception as e:
            msg = str(e)
            if any(code in msg for code in ("429", "500", "502", "503", "504", "UpstreamFailure")):
                time.sleep(min(backoff_base ** attempt, 30))
                continue
            break

    # HTTP
    attempt = 0
    while attempt < http_tries:
        attempt += 1
        try:
            resp = _http_search(q, limit, cursor)
            posts = (resp or {}).get("posts") or []
            next_cur = (resp or {}).get("cursor")
            return posts, next_cur, True
        except requests.HTTPError as e:
            status = getattr(e.response, "status_code", None)
            if status in _RETRYABLE_HTTP:
                time.sleep(min(backoff_base ** attempt, 30))
                continue
            raise
        except requests.RequestException:
            time.sleep(min(backoff_base ** attempt, 30))
            continue

    raise RuntimeError("Search failed after SDK and HTTP retries")

# ---------- CORE ----------
def count_mentions_last_n_days_for_terms(
    terms: List[str],
    days: int = 2,
    max_pages: int = 50,
    per_request_limit: int = 100,
    start_date: Optional[date] = None,   # NEW (optional)
    end_date: Optional[date] = None      # NEW (optional, exclusive)
) -> Counter:
    """
    Count unique posts per day for a list of search terms (aliases).
    - If start_date/end_date are given, uses that exact range (start inclusive, end exclusive).
    - Otherwise, uses the last `days` ending today.
    - Dedupe by post URI across terms and pages.
    """
    counts = Counter()

    # Build day list (NEW logic)
    if start_date and end_date:
        day_list = _date_range_between(start_date, end_date)
    else:
        today = datetime.now(timezone.utc).date()
        day_list = _date_range_last_n_days(days, today)

    # Normalize terms (quote multi-word)
    terms = [_normalize_term(t) for t in terms]

    for d in day_list:
        start_dt = datetime.combine(d, datetime.min.time(), tzinfo=timezone.utc)
        end_dt = datetime.combine(d + timedelta(days=1), datetime.min.time(), tzinfo=timezone.utc)

        seen_uris: set = set()
        daily_total = 0

        for base in terms:
            cursor = None
            pages = 0
            q_day = _q_for_day(base, d)

            while True:
                if pages >= max_pages:
                    break
                pages += 1

                try:
                    posts, cursor, used_http = _search_resilient(q_day, per_request_limit, cursor)
                except Exception:
                    # Give up this term/day, move to next term
                    break

                if not posts:
                    break

                first = posts[0]
                is_dict = isinstance(first, dict)

                if is_dict:
                    # HTTP
                    for post in posts:
                        uri = post.get("uri")
                        if not uri or uri in seen_uris:
                            continue
                        rec = post.get("record") or {}
                        dt = _parse_iso_created_at(rec.get("createdAt") or rec.get("created_at"))
                        if dt and (start_dt <= dt < end_dt):
                            seen_uris.add(uri)
                            daily_total += 1
                else:
                    # SDK
                    for post in posts:
                        uri = _get_post_uri(post)
                        if not uri or uri in seen_uris:
                            continue
                        dt = _post_created_at(post)
                        if dt and (start_dt <= dt < end_dt):
                            seen_uris.add(uri)
                            daily_total += 1

                if not cursor:
                    break

        counts[d] = daily_total

    return counts



In [2]:
# Code to count mentions of all company aliases in posts.
from datetime import datetime, timedelta, timezone, date
from collections import Counter
from typing import Dict, List, Iterable, Optional
import os, csv


def counts_for_companies(
    companies: Iterable[str],
    days: int = 2,
    start_date: Optional[date] = None,   # NEW
    end_date: Optional[date] = None      # NEW
) -> Dict[str, Counter]:
    out: Dict[str, Counter] = {}
    for name in companies:
        terms = COMPANY_ALIASES.get(name, [name])
        out[name] = count_mentions_last_n_days_for_terms(
            terms, days=days, start_date=start_date, end_date=end_date
        )
    return out

def write_company_date_matrix_csv(
    company_to_counts: Dict[str, Counter],
    days: int,
    save_dir: str,
    filename_prefix: str = "bluesky_counts_matrix",
    start_date: Optional[date] = None,   # NEW
    end_date: Optional[date] = None      # NEW
) -> str:
    os.makedirs(save_dir, exist_ok=True)

    # Build date header (NEW logic)
    if start_date and end_date:
        dates = _date_range_between(start_date, end_date)
        filename = f"{filename_prefix}_{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}.csv"
    else:
        today = datetime.now(timezone.utc).date()
        start_day = today - timedelta(days=days)
        dates = _date_range_last_n_days(days, today)
        filename = f"{filename_prefix}_{start_day.strftime('%Y%m%d')}-{today.strftime('%Y%m%d')}.csv"

    header = ["company"] + [d.isoformat() for d in dates]
    path = os.path.join(save_dir, filename)

    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for company, cnt in company_to_counts.items():
            row = [company] + [cnt.get(d, 0) for d in dates]
            writer.writerow(row)

    return path

# ---------- RUN ----------
if __name__ == "__main__":
    # EITHER use explicit dates (NEW):
    START_DATE = date(2024, 8, 1)   # inclusive
    END_DATE   = date(2025, 7, 31)   # exclusive
    DAYS = 2  # kept for compatibility; ignored when start/end provided

    company_counts = counts_for_companies(
        COMPANIES, days=DAYS, start_date=START_DATE, end_date=END_DATE
    )
    csv_path = write_company_date_matrix_csv(
        company_counts, DAYS, SAVE_DIR,
        start_date=START_DATE, end_date=END_DATE
    )

    print(f"Saved matrix to: {csv_path}\n")

    # Build the same date list for printing
    dates = _date_range_between(START_DATE, END_DATE)
    print("company,", ", ".join(d.isoformat() for d in dates))
    for c in COMPANIES:
        cnt = company_counts.get(c, Counter())
        row = [str(cnt.get(d, 0)) for d in dates]
        print(f"{c}, " + ", ".join(row))


Saved matrix to: /Users/nihan/Desktop/bluesky/bluesky_counts_matrix_20251001-20251003.csv

company, 2025-10-01, 2025-10-02
Walmart, 814, 757
Amazon, 2710, 1176
Apple, 4561, 4306


In [3]:
# Code to count mentions of containing ticker and/or stock name in posts. Not including posts with CEO name only.
from datetime import datetime, timedelta, timezone, date
from collections import Counter
from typing import Dict, List, Iterable, Optional
import os, csv

# Reuse helpers via import in your project, or paste the same helpers here:
# _date_range_last_n_days, _date_range_between, count_mentions_last_n_days_for_terms, TICKER_ALIASES, COMPANIES, SAVE_DIR

def counts_for_tickers(
    companies: Iterable[str],
    days: int = 2,
    start_date: Optional[date] = None,   # NEW
    end_date: Optional[date] = None      # NEW
) -> Dict[str, Counter]:
    out: Dict[str, Counter] = {}
    for name in companies:
        terms = TICKER_ALIASES.get(name, [name])
        out[name] = count_mentions_last_n_days_for_terms(
            terms, days=days, start_date=start_date, end_date=end_date
        )
    return out

def write_company_date_matrix_csv(
    tickers_to_counts: Dict[str, Counter],
    days: int,
    save_dir: str,
    filename_prefix: str = "bluesky_ticker_counts_matrix",
    start_date: Optional[date] = None,   # NEW
    end_date: Optional[date] = None      # NEW
) -> str:
    os.makedirs(save_dir, exist_ok=True)

    if start_date and end_date:
        dates = _date_range_between(start_date, end_date)
        filename = f"{filename_prefix}_{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}.csv"
    else:
        today = datetime.now(timezone.utc).date()
        start_day = today - timedelta(days=days)
        dates = _date_range_last_n_days(days, today)
        filename = f"{filename_prefix}_{start_day.strftime('%Y%m%d')}-{today.strftime('%Y%m%d')}.csv"

    header = ["company"] + [d.isoformat() for d in dates]
    path = os.path.join(save_dir, filename)

    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for company, cnt in tickers_to_counts.items():
            row = [company] + [cnt.get(d, 0) for d in dates]
            writer.writerow(row)

    return path

# ---------- RUN ----------
if __name__ == "__main__":
    START_DATE = date(2024, 8, 1)   # inclusive
    END_DATE   = date(2025, 7, 31)   # exclusive
    DAYS = 2  # kept for compatibility

    ticker_counts = counts_for_tickers(
        COMPANIES, days=DAYS, start_date=START_DATE, end_date=END_DATE
    )
    csv_path = write_company_date_matrix_csv(
        ticker_counts, DAYS, SAVE_DIR, start_date=START_DATE, end_date=END_DATE
    )

    print(f"Saved matrix to: {csv_path}\n")

    dates = _date_range_between(START_DATE, END_DATE)
    print("company,", ", ".join(d.isoformat() for d in dates))
    for c in COMPANIES:
        cnt = ticker_counts.get(c, Counter())
        row = [str(cnt.get(d, 0)) for d in dates]
        print(f"{c}, " + ", ".join(row))


Saved matrix to: /Users/nihan/Desktop/bluesky/bluesky_ticker_counts_matrix_20251001-20251003.csv

company, 2025-10-01, 2025-10-02
Walmart, 814, 757
Amazon, 2630, 1043
Apple, 4550, 4290


In [4]:
# Code to count mentions of CEO names only in posts (excluding posts with ticker or stock name ONLY).
from datetime import datetime, timedelta, timezone, date
from collections import Counter
from typing import Dict, List, Iterable, Optional
import os, csv

# Reuse helpers via import in your project, or paste the same helpers here:
# _date_range_last_n_days, _date_range_between, count_mentions_last_n_days_for_terms, CEO_LIST, COMPANIES, SAVE_DIR

def counts_for_ceos(
    companies: Iterable[str],
    days: int = 2,
    start_date: Optional[date] = None,   # NEW
    end_date: Optional[date] = None      # NEW
) -> Dict[str, Counter]:
    out: Dict[str, Counter] = {}
    for name in companies:
        terms = CEO_LIST.get(name, [name])
        out[name] = count_mentions_last_n_days_for_terms(
            terms, days=days, start_date=start_date, end_date=end_date
        )
    return out

def write_company_date_matrix_csv(
    ceo_to_counts: Dict[str, Counter],
    days: int,
    save_dir: str,
    filename_prefix: str = "bluesky_ceo_counts_matrix",
    start_date: Optional[date] = None,   # NEW
    end_date: Optional[date] = None      # NEW
) -> str:
    os.makedirs(save_dir, exist_ok=True)

    if start_date and end_date:
        dates = _date_range_between(start_date, end_date)
        filename = f"{filename_prefix}_{start_date.strftime('%Y%m%d')}-{end_date.strftime('%Y%m%d')}.csv"
    else:
        today = datetime.now(timezone.utc).date()
        start_day = today - timedelta(days=days)
        dates = _date_range_last_n_days(days, today)
        filename = f"{filename_prefix}_{start_day.strftime('%Y%m%d')}-{today.strftime('%Y%m%d')}.csv"

    header = ["company"] + [d.isoformat() for d in dates]
    path = os.path.join(save_dir, filename)

    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for company, cnt in ceo_to_counts.items():
            row = [company] + [cnt.get(d, 0) for d in dates]
            writer.writerow(row)

    return path

# ---------- RUN ----------
if __name__ == "__main__":
    START_DATE = date(2024, 8, 1)   # inclusive
    END_DATE   = date(2025, 7, 31)   # exclusive
    DAYS = 2  # kept for compatibility

    ceo_counts = counts_for_ceos(
        COMPANIES, days=DAYS, start_date=START_DATE, end_date=END_DATE
    )
    csv_path = write_company_date_matrix_csv(
        ceo_counts, DAYS, SAVE_DIR, start_date=START_DATE, end_date=END_DATE
    )

    print(f"Saved matrix to: {csv_path}\n")

    dates = _date_range_between(START_DATE, END_DATE)
    print("company,", ", ".join(d.isoformat() for d in dates))
    for c in COMPANIES:
        cnt = ceo_counts.get(c, Counter())
        row = [str(cnt.get(d, 0)) for d in dates]
        print(f"{c}, " + ", ".join(row))


Saved matrix to: /Users/nihan/Desktop/bluesky/bluesky_ceo_counts_matrix_20251001-20251003.csv

company, 2025-10-01, 2025-10-02
Walmart, 1, 1
Amazon, 82, 136
Apple, 15, 29
